#Chapter 14. Local effects and mutable state

### _more mature concept of 'referential transparency'_

## 14.1. Purely functional mutable state

- 순수 FP에서는 mutable state를 허용하지 않는다고 생각해 왔지만, 사실은 그렇지 않다!
- "참조 투명성"의 정의부터 살펴보자.
#### "참조 투명성(referential transparency)"의 정의
  * **어떠한 표현식 _e_가 프로그램 _p_안에서 쓰일 때,**
  * **모든 _e_가 나오는 부분을 _e_의 평가결과 값으로 교체해도 **
  * **_p_의 전체 의미가 달라지지 않을 때 _e_를 참조(에 대해) 투명하다고 한다.**
    * **모든 참조 투명한 x값에 대해 어떤 함수 f(x)가 참조 투명하면 f를 순수(pure)하다고 한다.**

 * **_pure_**한 quicksort 함수 정의

In [6]:
def quicksort(xs: List[Int]): List[Int] = if (xs.isEmpty) xs else {
    /* list => mutable array (swap, partitioning) => list */
    val arr = xs.toArray
    def swap(x: Int, y: Int) = {
      val tmp = arr(x)
      arr(x) = arr(y)
      arr(y) = tmp
    }
    def partition(l: Int, r: Int, pivot: Int) = {
      val pivotVal = arr(pivot)
      swap(pivot, r)
      var j = l
      for (i <- l until r) if (arr(i) < pivotVal) {
        swap(i, j)
        j += 1
      }
      swap(j, r)
      j
    }
    def qs(l: Int, r: Int): Unit = if (l < r) {
      val pi = partition(l, r, l + (r - l) / 2)
      qs(l, pi - 1)
      qs(pi + 1, r)
    }
    qs(0, arr.length - 1)
    arr.toList
  }

In [11]:
val mylist = List(3,1,2,5,7,9,0,4,8,6)

List(3, 1, 2, 5, 7, 9, 0, 4, 8, 6)

In [13]:
quicksort(mylist)

List(0, 1, 2, 3, 4, 5, 6, 7, 8, 9)

In [14]:
mylist

List(3, 1, 2, 5, 7, 9, 0, 4, 8, 6)

<img src="files/imgs/01.png">

##14.2. A data type to enforce scoping of side effects

- quicksort() 함수에서 입력 리스트를 직접 바꿀 수도 있었을 것이다. 그랬다면 외부에서 side effect 관찰 가능.
- 스칼라의 타입시스템을 활용해서 side-effect의 스코프를 (local로) 강제하는 데이터 타입을 만들어보자.
  * **_성능과 표기상의 편의를 생각하면 이런 짓을 하는 비용이 크다. 쓸지 말지는 알아서 결정할 것.._**

###14.2.1. A little language for scoped mutation

- State[S,A] 모나드
  - mutable state를 다루기 편한 타입임.
  - S => (A,S) 함수였음을 기억할 것.
  - 